# Tfimps calculations for transverse field Ising

In [1]:
import sys
import tensorflow as tf
import numpy as np
sys.path.append("../")
from tfimps import Tfimps

In [3]:
phys_d = 2
bond_d = 16

# Pauli matrices. For now we avoid complex numbers
X = tf.constant([[0,1],[1,0]], dtype=tf.float64)
iY = tf.constant([[0,1],[-1,0]], dtype=tf.float64)
Z = tf.constant([[1,0],[0,-1]], dtype=tf.float64)

I = tf.eye(phys_d, dtype=tf.float64)

XX = tf.einsum('ij,kl->ikjl', X, X)
YY = - tf.einsum('ij,kl->ikjl', iY, iY)
ZZ = tf.einsum('ij,kl->ikjl', Z, Z)
X1 = (tf.einsum('ij,kl->ikjl', X, I) + tf.einsum('ij,kl->ikjl', I, X)) / 2

In [4]:
# Ising Hamiltonian (at criticality). Exact energy is -4/pi=-1.27324...
h_ising = - ZZ - X1

imps = Tfimps(phys_d, bond_d, symmetrize=True, hamiltonian=h_ising)

train_op = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(imps.variational_e)

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    for i in range(50):
        print(sess.run([imps.variational_e, train_op])[0])

-1.0243753723737807
-1.0351221694985309
-1.04809965905192
-1.0632659302948249
-1.0804973908404347
-1.099556064680165
-1.1200891772621728
-1.1416123122595294
-1.1634928724211033
-1.184964824670064
-1.2051748756762435
-1.2232494975372095
-1.2383762686420967
-1.2499005541464878
-1.2574356981860701
-1.2609722647097255
-1.2609525913335844
-1.258260493211469
-1.2540872012483655
-1.249688027370728
-1.2461139633456084
-1.2440294055618044
-1.2436719967886642
-1.2449255941088435
-1.2474412541716773
-1.2507565842835542
-1.2543918326025891
-1.2579188967716184
-1.2610057774500036
-1.2634395338245625
-1.2651302402359352
-1.2660986707890176
-1.2664516609762415
-1.266350522614445
-1.2659784225655948
-1.2655118510631642
-1.2650994476165904
-1.2648492727338052
-1.264823868435943
-1.2650415072536931
-1.2654818246326327
-1.266094289283879
-1.2668083631316025
-1.2675445117022084
-1.2682253380493327
-1.2687860239727935
-1.2691830531907238
-1.2693999999175576
-1.269449200238813
-1.2693685446564265


## AKLT 

In [2]:
phys_d = 3
bond_d = 2

# Follow Annals of Physics Volume 326, Issue 1, Pages 96-192.
# Note that even though the As are not symmetric, the transfer matrix is.

Aplus = np.array([[0, 1/np.sqrt(2)], [0, 0]])
Aminus = np.array([[0, 0], [-1/np.sqrt(2), 0]])
A0 = np.array([[-1/2, 0], [0, 1/2]])
A_matrices = np.array([Aplus, A0, Aminus])

# Spin 1 operators.

X = tf.constant([[0, 1, 0 ], [1, 0, 1], [0, 1, 0]], dtype=tf.float64) / np.sqrt(2)
iY = tf.constant([[0, -1, 0 ], [1, 0, -1], [0, 1, 0]], dtype=tf.float64) / np.sqrt(2)
Z = tf.constant([[1, 0, 0], [0, 0, 0], [0, 0, -1]], dtype=tf.float64)

XX = tf.einsum('ij,kl->ikjl', X, X)
YY = - tf.einsum('ij,kl->ikjl', iY, iY)
ZZ = tf.einsum('ij,kl->ikjl', Z, Z)

hberg = XX + YY + ZZ
h_aklt = hberg + tf.einsum('abcd,cdef->abef', hberg, hberg) / 3

aklt = Tfimps(phys_d, bond_d, A_matrices, symmetrize=False, hamiltonian=h_aklt)

In [6]:
train_op = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(aklt.variational_e)

with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())

    for i in range(100):
        print(sess.run([aklt.variational_e, train_op])[0])

-0.6666666666666666


InvalidArgumentError: Self-adjoint eigen decomposition was not successful. The input might not be valid.
	 [[Node: SelfAdjointEigV2 = SelfAdjointEigV2[T=DT_DOUBLE, compute_v=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape)]]

Caused by op 'SelfAdjointEigV2', defined at:
  File "/Users/austen/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/austen/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/austen/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/austen/anaconda/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/austen/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File